In [1]:
#
# Importing all necessary libraries. We shall use ICP registration and other functions
# (such mesh handler, visualization) provided in Open3D. Also some speed-ups with JIT.
#
import open3d as o3d
import numpy as np
import scipy.spatial as sp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import time
import csv
import copy
from numba import jit

In [2]:
#
# Barycenter of a point cloud A
#
@jit(target_backend='mps')
#
def bar(A):
    #
    return np.sum(A, axis=1)/A.shape[1]

In [3]:
#
# Centering a point cloud A
#
@jit(target_backend='mps')
#
def barycentered(A):
    #
    bar = np.sum(A, axis=1)/A.shape[1]
    #
    return A - bar[:, np.newaxis]

In [4]:
#
# Detecting nearest neighbours between two unlabelled point coluds A and B
#
# Assumption: cardinality of A <= cardinality of B 
#
# Input: point clouds A, B
#
# Output: for each point of A a matching point of B, as a 0/1-matrix
#
@jit(target_backend='mps')
#
def nearest_neighbours(A, B):
    #
    na = A.shape[1]
    nb = B.shape[1]
    #
    assert na <= nb
    #
    tree = sp.KDTree(A.T, leafsize=10, compact_nodes=True, copy_data=True, balanced_tree=True)
    #
    matching = []
    I = np.identity(na)
    #
    for i in range(nb):
        _, ind = tree.query(B.T[i], k=1, p=2, workers=-1)
        matching += [I[ind]]
    #    
    return np.array(matching)

In [5]:
#
# Input: point cloud P, point cloud Q, of dimension d=3, with n points in Q
#
# Assumption: cardinality of P <= cardinality of Q
#
# Algorithm: generate a random orthogonal transform O from O(d), 
# a random permutation matrix S from Sym(n), and replace Q with O*Q*S. 
# Then produce the initialization O_init
#
# Output: O and O_init
#
# Key function for sorting solutions (jit does not take lambda declarations)
#
@jit(target_backend='mps')
#
def key(x):
    return x[2]
#
# test_point_cloud GPU speed-up
#
@jit(target_backend='mps')
#
def init_point_clouds(P, Q, tol=0.05, verbose=False):
    #
    dim = dim_p = P.shape[0]
    num_p = P.shape[1]
    #
    num_q = Q.shape[1]
    #
    assert(dim_p == Q.shape[0] == 3)
    assert(num_p <= num_q)
    #
    normP = np.linalg.norm(P, 2)
    normQ = np.linalg.norm(Q, 2)
    #
    if verbose:
        print("Number of points in sample 1:   {}".format(num_p))
        print("Number of points in sample 2:   {}".format(num_q))
    #
    seed = np.random.normal(0.0, 1.0, (dim, dim))
    O = np.linalg.qr(seed, mode='complete')[0]
    #
    S = np.random.default_rng().permutation(np.identity(num_q))
    #
    P = barycentered(P)
    #
    Q0 = copy.deepcopy(Q)
    Q0 = barycentered(Q0)
    #
    Q  = O @ Q @ S
    Q = barycentered(Q)
    #
    Ep = P @ P.T
    Eigp, Up = np.linalg.eigh(Ep)
    #
    Eq = Q @ Q.T
    Eigq, Uq = np.linalg.eigh(Eq)
    #
    U0 = Uq @ Up.T
    #
    isoms_discrete = [[1,1,1],[-1,1,1],[1,-1,1],[1,1,-1],
                      [1,-1,-1],[-1,1,-1],[-1,-1,1],[-1,-1,-1]]
    isoms_discrete = [np.diag(m) for m in isoms_discrete]
    #
    sols = []
    for isom in isoms_discrete:
        U = U0 @ Up @ isom @ Up.T
        nn = nearest_neighbours(U @ P, Q)
        diff_init = U @ P - Q @ nn
        d_init = np.linalg.norm(diff_init, 2) / np.sqrt(normP * normQ)
        sols += [(U, nn, d_init)]
    sols = sorted(sols, key=key)
    #
    O_init, S_init, dist_init = sols[0]
    #
    if verbose:
        print("Original orthogonal transform applied: ")
        print(O)
        print("Initialization orthogonal transform:")
        print(O_init)
        diff = np.linalg.norm(O - O_init, 2)
        print("Difference between transforms: {}".format(diff))
    #
    return O, O_init

In [6]:
#
# Draw ICP registration results
#
@jit(target_backend='mps')
#
def draw_registration_result(source, target, transformation):
    #
    source_temp = copy.deepcopy(source)
    source_temp.compute_vertex_normals()
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp = copy.deepcopy(target)
    target_temp.compute_vertex_normals()
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw([source_temp, target_temp])
    #

In [7]:
#
# Comparing ICP results without initialization (initial transform = identity) 
# to ICP results with E-init initialization (initial transform = E-init output)
#
@jit(target_backend='mps')
#
def compare_icp_init(source_mesh_1, source_mesh_2, sample_size_1=20000, sample_size_2=20000, icp_threshold=0.001, icp_max_iter=1000):    
    #
    mesh_1 = copy.deepcopy(source_mesh_1)
    mesh_2 = copy.deepcopy(source_mesh_2)
    #
    c_mesh_1 = mesh_1.get_center()
    mesh_1.translate(-1.0*c_mesh_1)
    assert np.allclose(mesh_1.get_center(), np.zeros((1,3)))
    #
    c_mesh_2 = mesh_2.get_center()
    mesh_2.translate(-1.0*c_mesh_2)
    assert np.allclose(mesh_2.get_center(), np.zeros((1,3)))
    #
    pcl_mesh_1 = mesh_1.sample_points_poisson_disk(number_of_points=sample_size_1)
    pcl_mesh_2 = mesh_2.sample_points_poisson_disk(number_of_points=sample_size_2)
    #
    O, O_init =\
    init_point_clouds(np.asarray(pcl_mesh_1.points).T, np.asarray(pcl_mesh_2.points).T, verbose=False)
    #
    #
    r_mesh_1 = copy.deepcopy(mesh_1)
    r_mesh_2 = copy.deepcopy(mesh_2)
    r_mesh_2.rotate(O)
    #
    source = copy.deepcopy(pcl_mesh_1)
    target = copy.deepcopy(pcl_mesh_2)
    target.rotate(O)
    #
    init_id = np.identity(4)
    vec_kw  = O_init@(c_mesh_1 - c_mesh_2)
    init_kw = np.c_[np.c_[O_init, vec_kw].T, [0,0,0,1]].T
    transform_init_array = [init_id, init_kw]
    #
    # Necessary for Point-to-Plane ICP; comment for Point-to-Point ICP
    #
    target.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=20)
    )
    # Set the ICP parameters
    icp_criteria = o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=icp_max_iter,
                                                                     relative_fitness=1e-12,
                                                                     relative_rmse=1e-12)

    #
    for transform_init in transform_init_array:
        #
        # Using Point-to-Plane ICP 
        #
        reg_p2p = o3d.pipelines.registration.registration_icp(
                  source, target, icp_threshold, transform_init,
                  o3d.pipelines.registration.TransformationEstimationPointToPlane(),
                  icp_criteria)
        #
        # Uncomment for Point-to-Point ICP
        #
        #reg_p2p = o3d.pipelines.registration.registration_generalized_icp(
        #          source, target, icp_threshold, transform_init,
        #          o3d.pipelines.registration.TransformationEstimationForGeneralizedICP(),
        #          icp_criteria)
        #
        print("ICP initialization transform:")
        print(transform_init)
        print("ICP registration transform:")
        print(reg_p2p.transformation)
        #
        draw_registration_result(r_mesh_1, r_mesh_2, reg_p2p.transformation)

In [8]:
#
# Loading the 3D scan of "Enfant au Chien" statue: before restoration (broken) 
# and after (major defects repaired and gaps filled in)
#
mesh_Enfant_1 = o3d.io.read_triangle_mesh('Enfant_au_chien-Broken.obj')
#
mesh_Enfant_2 = o3d.io.read_triangle_mesh('Enfant_au_chien.obj')

In [9]:
#
# Producing one interactive exhibit for registration without ICP intialization. 
# After the window is closed, a second interactive exhibit is produced, this times
# for registration with initialized ICP. Parameters can be changed and tuned as needed. 
#
compare_icp_init(mesh_Enfant_1, mesh_Enfant_2, 20000, 20000, 0.001, 400)

ICP initialization transform:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
ICP registration transform:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


[error] GLFW error: Cocoa: Failed to find service port for display


FEngine (64 bits) created at 0x120c00000 (threading is enabled)
FEngine resolved backend: OpenGL
ICP initialization transform:
[[-0.04000548 -0.33530051 -0.94126146  0.00254177]
 [ 0.28360356 -0.90708493  0.31107227 -0.04142781]
 [-0.95810677 -0.25450051  0.13138078 -0.00535805]
 [ 0.          0.          0.          1.        ]]
ICP registration transform:
[[-0.03938634 -0.33729389 -0.94057512  0.00157537]
 [ 0.28181894 -0.90684133  0.31339573 -0.03949055]
 [-0.95865885 -0.25272837  0.13077297 -0.00305059]
 [ 0.          0.          0.          1.        ]]


In [10]:
#
# Loading the 3D scan of "Hermanubis" statue: before restoration (broken) 
# and after (major defects repaired and gaps filled in)
#
mesh_Hermanubis_1 = o3d.io.read_triangle_mesh('Hermanubis_broken.stl')
#
mesh_Hermanubis_2 = o3d.io.read_triangle_mesh('Hermanubis_restored.stl')

In [11]:
#
# Producing one interactive exhibit for registration without ICP intialization. 
# After the window is closed, a second interactive exhibit is produced, this times
# for registration with initialized ICP. Parameters can be changed and tuned as needed. 
#
compare_icp_init(mesh_Hermanubis_1, mesh_Hermanubis_2, 10000, 10000, 0.001, 400)

ICP initialization transform:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
ICP registration transform:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
ICP initialization transform:
[[-0.45462251 -0.88954535 -0.04502727 -0.36541218]
 [ 0.5783189  -0.25636001 -0.77448486  0.02892199]
 [ 0.67739621 -0.37813837  0.63098792 -0.09488898]
 [ 0.          0.          0.          1.        ]]
ICP registration transform:
[[-0.45462251 -0.88954535 -0.04502727 -0.36541218]
 [ 0.5783189  -0.25636001 -0.77448486  0.02892199]
 [ 0.67739621 -0.37813837  0.63098792 -0.09488898]
 [ 0.          0.          0.          1.        ]]


In [12]:
#
# Loading the 3D scan of "Pan and Bacchus" statue: before restoration (broken) 
# and after (major defects repaired and gaps filled in)
#
mesh_Pan_1 = o3d.io.read_triangle_mesh('Pan_broken.obj')
#
mesh_Pan_2 = o3d.io.read_triangle_mesh('Pan_restored.obj')

In [13]:
#
# Producing one interactive exhibit for registration without ICP intialization. 
# After the window is closed, a second interactive exhibit is produced, this times
# for registration with initialized ICP. Parameters can be changed and tuned as needed. 
#
compare_icp_init(mesh_Pan_1, mesh_Pan_2, 15000, 20000, 0.001, 400)

ICP initialization transform:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
ICP registration transform:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


[error] GLFW error: Cocoa: Failed to find service port for display


ICP initialization transform:
[[ -0.71674603   0.68275654  -0.14184018 -15.61555164]
 [ -0.65822297  -0.59524017   0.46090309  16.7360043 ]
 [  0.23025563   0.42371292   0.87604207 -10.67696183]
 [  0.           0.           0.           1.        ]]
ICP registration transform:
[[ -0.71674603   0.68275654  -0.14184018 -15.61555164]
 [ -0.65822297  -0.59524017   0.46090309  16.7360043 ]
 [  0.23025563   0.42371292   0.87604207 -10.67696183]
 [  0.           0.           0.           1.        ]]
